#### Llama2 7B torch.float16 추론 
> * Llama2 모델은 GPU T4x16G 에서는 사용할 수 없다. 공식문서에서는 T4x27G 이상 사용을 권하고 있다.
```
1. casllm-base-7b-hf: Meta Llama2 7B
2. parameter data type: touch.float16
3. qutanization: not quantized
4. processor: cpu
5. write type: pytorch bin
6. transformers: LlamaForCausalLM, LlamaTokenizer
```

In [1]:
import torchLlamaForCausalLM, LlamaTokenizer
from transformers import 

In [2]:
# KoAlpaca-ployglot 12.8B (Non 양자화 모델)
# 파일형식: Hugging Transformers 의 Safe serialization 모델
# 파일크기: 1G X 28개
model_id = "/data/bwllm/models/casllm-base-7b-hf"
model = LlamaForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")
tokenizer = LlamaTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
# 모델의 첫 번째 파라미터의 데이터 타입 확인
first_param_dtype = next(model.parameters()).dtype
print(f"First parameter data type: {first_param_dtype}")

First parameter data type: torch.float16


In [4]:
# 양자화 여부 확인
if 'quantized' in str(first_param_dtype):
    print("The model is quantized.")
else:
    print("The model is not quantized.")

The model is not quantized.


In [ ]:
"""
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("GPU is not available. Using CPU.")

# 장치 확인   
print(device)

# 모델을 해당 디바이스로 이동
model.to(device)
"""

In [5]:
def make_answer(prompt_status, question):
    
    messages = prompt_status + [ {"role": "질문", "content": question}]    
    prompt_text = "\n".join(
        [f"### {msg['role']}:\n{msg['content']}" for msg in messages]
    )
    return prompt_text

In [6]:
prompt_status = [
            {
                "role": "맥락",
                "content": "KoAlpaca(코알파카)는 EleutherAI에서 개발한 Polyglot-ko 라는 한국어 모델을 기반으로, 자연어 처리 연구자 Beomi가 개발한 모델입니다.",
            },
            {
                "role": "맥락",
                "content": "ChatKoAlpaca(챗코알파카)는 KoAlpaca를 채팅형으로 만든 것입니다.",
            },
            {"role": "명령어", "content": "친절한 AI 챗봇인 ChatKoAlpaca 로서 답변을 합니다."},
            {
                "role": "명령어",
                "content": "인사에는 짧고 간단한 친절한 인사로 답하고, 아래 대화에 간단하고 짧게 답해주세요.",
            },
        ]

prompt_status2 = []

question = "정보화시스템 감리 절차 과정에 대해 설명해줘"
prompt_text = make_answer(prompt_status2, question)

print(prompt_text)

### 질문:
정보화시스템 감리 절차 과정에 대해 설명해줘


In [7]:
# 토크나이저를 사용해 입력 텍스트 인코딩
inputs = tokenizer.encode(prompt_text, return_tensors='pt')

In [8]:
# 모델을 사용해 문장 생성
# num_return_sequences는 반환할 문장 수, max_length는 최대 길이를 설정
output_sequences = model.generate(
    input_ids=inputs,
    max_length=2000,
    num_return_sequences=1,
    temperature=1.0,  # 생성 다양성 조절
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/data/bwllm/venv/llama/lib/python3.10/site-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [9]:
print(output_sequences)

tensor([[    1,   835, 29871,  ..., 29889,    13,    13]])


In [10]:
# 모델을 GPU로 이동
# T4 16G 에서 메모리에 로딩하는 것을 실패함.
#model.to('cuda')

# 입력 데이터도 GPU로 이동
#inputs = inputs.to('cuda')

print(inputs)

tensor([[    1,   835, 29871,   239,   170,   139, 31406, 29901,    13, 30852,
         31199, 31225, 30889, 30784,   240,   136,   159, 29871,   237,   179,
           147, 30826, 29871,   239,   163,   139, 31817, 29871, 31906, 30852,
         31054, 29871, 30890, 31435, 29871,   239,   135,   167, 31976, 31435,
           239,   167,   155]])


In [11]:
# 생성된 문장을 디코딩하여 출력
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print(generated_text)

### 질문:
정보화시스템 감리 절차 과정에 대해 설명해줘요.

### 답변:

1. 정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

정보화시스템 감리 절차 과정에 대해 설명해줘요.

